In [1]:
 echo $PS1

[PEXP\[\]ECT_PROMPT>


In [2]:
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [3]:
TMP=/tmp/test-unix-alias

BIN_DIR=$PWD/..

alias | wc -l

temp_dir=$TMP/test-2899

mkdir -p "$temp_dir"
cd "$temp_dir"
pwd


0
/tmp/test-unix-alias/test-2899


In [4]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

0
0


In [5]:
function group-members () { ypcat group | $GREP -i "$1"; }

In [6]:
## ERROR I - group-members

# $ group-members
# | ypcat: can't get local yp domain: Local domain name not set
# | bash: -i: command not found

In [7]:
## TEST 1 - do-make
function do-make () { /bin/mv -f _make.log _old_make.log; make "$@" >| _make.log 2>&1; $PAGER _make.log; }

In [8]:
## 1A) do-make
rm -rf ./*
mkdir testfolder
echo "Hi Mom!" > himom.txt
echo "Hi Dad!" > hidad.txt

do-make "himom.txt"

cat _make.log
do-make "hidad.txt"
cat _make.log

ls -l

/bin/mv: cannot stat '_make.log': No such file or directory
make: Nothing to be done for 'himom.txt'.
make: Nothing to be done for 'himom.txt'.
make: Nothing to be done for 'hidad.txt'.
make: Nothing to be done for 'hidad.txt'.
total 20
-rw-rw-r-- 1 aveey aveey    8 Oct  6 22:12 hidad.txt
-rw-rw-r-- 1 aveey aveey    8 Oct  6 22:12 himom.txt
-rw-rw-r-- 1 aveey aveey   42 Oct  6 22:12 _make.log
-rw-rw-r-- 1 aveey aveey   42 Oct  6 22:12 _old_make.log
drwxrwxr-x 2 aveey aveey 4096 Oct  6 22:12 testfolder


In [9]:
## TEST 2 - MERGING ALIASAS & FUNCTIONS

# $ man merge
#   merge [ options ] file1 file2 file3
#   merge  incorporates all changes that lead from file2 to file3 into file1.
# NOTE: merge -p mod-file1 original mod-file2 >| new-file
alias merge='echo "do-merge MODFILE1 OLDFILE MODFILE2 > NEWFILE"'
alias do-merge='/usr/bin/merge -p'
# note: version of merge usinf diff3 to specify diff program (shell wrapper with whitespace ignored)
alias diff3-merge='/usr/bin/diff3 --merge --text --diff-program=diff.sh'
## TODO: --auto
function kdiff-merge() {
    if [ "$3" = "" ]; then
        echo "usage: $0 changed1 old changed2 output"
        return
    fi
    kdiff3 --merge --output "$4" "$1" "$2" "$3"
}

In [10]:
# 2A) merge REPLACED BY do-merge 

## INITIAL FILES TO TEST THE ALIASES
printf "THIS IS THE 1ST FILE." > tf1.txt
printf "THIS IS THE 2ND FILE." > tf2.txt
printf "THIS IS THE 3RD FILE." > tf3.txt

# MESSAGE SHOWN IF MERGE IS USED
merge tf1.txt tf2.txt

do-merge MODFILE1 OLDFILE MODFILE2 > NEWFILE tf1.txt tf2.txt


In [11]:
# 2B) do-merge

## ERROR II - do-merge DOESN'T SEEM TO WORK
# $ do-merge tf1.txt tf2.txt tf3.txt
# | bash: /usr/bin/merge: No such file or directory

In [12]:
# 2C) diff3-merge

## ERROR III - Undefined variable (TMP)

# /usr/bin/diff3 --help

# $ diff3-merge tf1.txt tf2.txt tf2.txt -A

# | TMP: Undefined variable.
# | TMP: Undefined variable.
# | THIS IS THE 1ST FILE.

In [13]:
# 2D) kdiff-merge

#kdiff-merge "abc" "def" "ghi" "himom.txt"
## ERROR IV - kdiff-merge NOT WORKING AS kdiff3 COMMAND MISSING
# $ kdiff-merge abc def ghi
# | bash: kdiff3: command not found

In [14]:
# TEST 3 - WHICH FUNCTION
function which { /usr/bin/which "$@" 2> /dev/null; }
#
# full-dirname(filename): returns full path of directory for file
#
function full-dirname () { local dir=$(dirname $1); case $dir in .*) dir="$PWD/$1";; esac; echo $dir; }
#
# base-name-with-dir(file, suffix): version of basename include dir
function basename-with-dir {
    local file="$1"
    local suffix="$2"
    echo $(dirname "$file")/$(basename "$file" "$suffix");
}


In [15]:
### TO FIX THE ERROR USE "himom.txt" --> himom.txt ###

# 3A) full-dirname RETURNS THE FULL PATH OF THE FILE
full-dirname himom.txt

# 3B) base-name-with-dir RETURNS THE BASENAME INCLUDING dir
basename-with-dir himom.txt

/tmp/test-unix-alias/test-2899/himom.txt
./himom.txt


In [16]:
## 3C) rpm-extract IS USED FOR RED HAT BASED DISTROS (RHEL, Fedora) 
function rpm-extract () { rpm2cpio "$1" | cpio --extract --make-directories; }

# # ERROR V:
# $ rpm-extract
# | bash: rpm2cpio: command not found
# | cpio: premature end of archive


In [17]:
# TEST 4 - DEVICE INFO ALIASES
alias dump-url='wget --recursive --relative'
#
alias gtime='/usr/bin/time'

# see 'man 5 os-release'
alias linux-version="cat /etc/os-release"
alias os-release=linux-version
alias system-status='system_status.sh -'
#

In [18]:
## 4A) dump-url GIVES ERROR FOR BATSPP 2.1.X 
#dump-url https://www.example.com/

In [19]:
## 4B) gtime
# ERROR - VI 

# $ gtime
# | /usr/bin/time: missing program to run
# | Try '/usr/bin/time --help' for more information


In [20]:
## 4C) linux-version GIVES INFO ALL INFO ABOUT THE OS INSTALLED
linux-version
# os-release ALSO WORKS SAME AS linux-version
os-release

PRETTY_NAME="Ubuntu 22.04.1 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.1 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
PRETTY_NAME="Ubuntu 22.04.1 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.1 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [21]:
## 4D) system-status - ERROR (STUCK DUE TO OSTYPE: Undefined variable)

# $ system-status (# STUCK DUE TO OSTYPE: Undefined variable.)
# | Linux ASPIRE5742GLUBU 5.15.0-47-generic #51-Ubuntu SMP Thu Aug 11 07:51:15 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
# | OSTYPE: Undefined variable.


In [22]:
## 5A)
# propos-command: show apropos output for lExecutable programs or shell commands (i.e., man section 1)
# TODO: use '$@' for '$*' (or note why not appropriate)
# TODO: 'apropos --section 1'
# EX: apropos-command time | grep asctime | wc -l => 0
function apropos-command () { apropos $* 2>&1 | $GREP '(1)' | $PAGER; }
#
# 
function split-tokens () { perl -pe "s/\s+/\n/g;" "$@"; }
alias tokenize='split-tokens'
#

In [23]:
# tokenize SPLITS THE CONTENTS IN SEVERAL LINES
tokenize _old_make.log

make:
Nothing
to
be
done
for
'himom.txt'.


In [24]:
# 5B)
# perl-echo(arg1): print arg1 via perl
# NOTE: Usually Bash $'string' special tokens can be used instead
#       (e.g., perl-echo "A\tB" => echo $'A\tB').
# TODO: try to minimize use of quotes in perl-echo (e.g., need to mix single with double) due to bash quirks
function perl-echo () { perl -e 'print "'$1'\n";'; }
## MISC
function perl-echo-sans-newline () { perl -e 'print "'$1'";'; }
#

In [25]:
# perl-echo ONLY PRINTS OUT THE 1ST ARGUMENT
PERL_PRINT='This is Ubuntu!'
perl-echo $PERL_PRINT
perl-echo-sans-newline $PERL_PRINT

This
This

In [26]:
# 5C)
# perl-printf(format, arg): output ARG2 via perl format ARG1
## TODO: generalize perl-printf to more than 2 args
## function perl-printf () { perl -e 'printf "$1\n", @_[1..$#_];';  }
##
## function perl-printf () { perl -e "printf \"$1\"", qw/$2 "$3" "$4" "$5" "$6" "$7" "$8" "$9"/; }
## MISC
function perl-printf () { perl -e "printf \"$1\", $2;"; }
##
## TODO: get following to work for 'perl-print "how now\nbrown cow\n"'
## function perl-print () { perl -e "print $1"; -e 'print "\n";'; }
## MISC
function perl-print () { perl -e "printf \"$1\";" -e 'print "\n";'; }
## MISC
function perl-print-n () { perl -e "printf \"$1\";"; }
#
# quote-tokens(): puts double quote around each token on commnd line
# note: used to circumvent google search's annoying search term dropping
function quote-tokens () { echo "$@" | perl -pe 's/(\S+)/"\1"/g;'; }

In [27]:
perl-printf 'ONE KISS IS ALL IT TAKES\n'
perl-print '2\n3\n5\n7\n11'
perl-print-n 'A B C D E F G\n'
quote-tokens 'HELP ME!'

ONE KISS IS ALL IT TAKES
2
3
5
7
11
A B C D E F G
"HELP" "ME!"


In [28]:
# 6A)
# Unix/Win32 networking aliases
if [ "$OSTYPE" != "cygwin" ]; then alias ipconfig=ifconfig; fi
alias set-display-local='export DISPLAY=localhost:0.0'

set-display-local
echo $DISPLAY

localhost:0.0


In [29]:
alias bash-trace-on='set -o xtrace'
alias bash-trace-off='set - -o xtrace'

function trace-cmd() {
    (
        echo 'start: $(date)';
        bash-trace-on; 
        eval '$@'; 
        bash-trace-off;
        echo 'end: $(date)';
    ) 2>&1 | $PAGER;
}
alias cmd-trace='trace-cmd'


In [30]:
## bash-trace-on ENABLES bash tracing
bash-trace-on
## trace-cmd TRACES COMMANDS AND STATS (TIME) OF THE USER
trace-cmd
## bash-trace-on DISABLES bash tracing
bash-trace-off
trace-cmd

+ trace-cmd
+ cat
+ echo 'start: $(date)'
start: $(date)
+ set -o xtrace
+ eval '$@'
+ set - -o xtrace
end: $(date)
+ set - -o xtrace
start: $(date)
+ eval '$@'
+ set - -o xtrace
end: $(date)


In [31]:
function cmd-usage () {
    local command="$@";
    local usage_file="_$(echo "$command" | tr ' ' '_')-usage.list"
    $command --help  2>&1 | ansifilter > "$usage_file"
    $PAGER_NOEXIT "$usage_file"
}

In [32]:
function compress-dir() {
    log_file=$TEMP/compress_$(basename "$1").log
    find "$1" \( -not -type l \) -exec gzip -vf {} \; >| "$log_file" 2>&1 
    $PAGER "$log_file"
}

function uncompress-dir() {
    log_file=$TEMP/uncompress_$(basename "$1").log;
    find "$1" \( -not -type l \) \( -not -name \*.tar.gz \) -exec gunzip -vf {} \; >| "$log_file" 2>&1; $PAGER "$log_file";
}
alias compress-this-dir='compress-dir $PWD'
alias ununcompress-this-dir='uncompress-dir $PWD'

In [33]:
## ERROR (PERMISSION DENIED - NEED TO OBSERVE)
# $ ls
# $ cp tf2.txt testfolder/
# $ linebr
# $ compress-this-dir ./testfolder
# $ linebr
# $ ls
# $ linebr
# $ ununcompress-this-dir ./testfolder

In [34]:
## TEST 9B)

alias old-count-exts='$LS | count-it "\.[^.]*\w" | sort $SORT_COL2 -rn | $PAGER'
function count-exts () { $LS | count-it '\.[^.]+$' | sort $SORT_COL2 -rn | $PAGER; }

alias kill-iceweasel='kill_em.sh iceweasel'

In [35]:
# ERROR GENERATED: count-it NOT MENTIONED IN THIS TEST
# $ old-count-exts
# | bash: count-it: command not found

# $ kill-iceweasel
# | pattern=/:[0-9][0-9] [^ ]*iceweasel/
# | filter=/($)(^)/
# | OSTYPE: Undefined variable.

#kill-iceweasel

In [36]:
## EXCEPTION ERROR FOR TOKEN - BATSPP 2.1.X

## cmd-usage(command): output usage for command to _command.list (with spaces replaced by underscores)

function cmd-usage () {
    local command="$@"
    local usage_file="_$(echo "$command" | tr ' ' '_')-usage.list"
    $command --help  2>&1 | ansifilter > "$usage_file"
    $PAGER_NOEXIT "$usage_file"
}


In [37]:
## cmd-usage (HELP REQUIRED)

In [38]:
echo "Done!"

Done!
